In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing import sequence
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [102]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import SimpleRNN,Dense,Input,Embedding

In [137]:
men=pd.read_csv("Indian-Male-Names.csv")
female=pd.read_csv("Indian-Female-Names.csv")

In [138]:
men.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [139]:
men.shape,female.shape

((14845, 3), (15382, 3))

In [140]:
total=[]

In [141]:
total=men[:]

In [142]:
total=total.append(female[:])

In [143]:
total.shape

(30227, 3)

In [144]:
total.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [145]:
###   total.sample(frac=1).reindex(drop=True)    ## to suffle one method
total=shuffle(total)

In [146]:
total.reset_index(drop=True,inplace=True)

In [147]:
#total

In [148]:
X,y=total.iloc[:,0].values,total.iloc[:,1].values

In [149]:
RegexpTokenizer.tokenize

<function nltk.tokenize.regexp.RegexpTokenizer.tokenize(self, text)>

In [150]:
def fix_Plan(location):
    
    letters_only = re.sub(r"\^w+",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(location))
    return letters_only

In [151]:
X_final=[]
for ind in X:
    X_final.append(fix_Plan(ind))

In [152]:
X

array(['khushi', 'saddham', 'raj kishor', ..., 'naresh', 'meethu sarkar',
       'aarti jain'], dtype=object)

In [153]:
import string

In [154]:
lit=string.ascii_lowercase
vocab=dict(zip(lit,range(3,29)))

In [155]:
X_mod=[]
for mod in X_final:
    gen=[vocab[index] for index in mod.lower() if index in vocab]
    gen.insert(0,1)
    gen.append(2)
    X_mod.append(gen)

In [156]:
X_padded=sequence.pad_sequences(X_mod,maxlen=15)

In [157]:
X_padded.shape

(30227, 15)

In [158]:
y=LabelEncoder().fit_transform(y)

In [159]:
X_train, X_test, y_train, y_test = train_test_split(
...     X_padded, y, test_size=0.33, random_state=42)

In [160]:
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [161]:
in_layer=Input(shape=(15,))
embad=Embedding(input_dim=29,output_dim=3)(in_layer)
rnn=SimpleRNN(units=60)(embad)
out=Dense(units=1,activation="sigmoid")(rnn)

In [162]:
model=Model(inputs=in_layer,outputs=out)

In [163]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 15)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 3)             87        
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 60)                3840      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 61        
Total params: 3,988
Trainable params: 3,988
Non-trainable params: 0
_________________________________________________________________


In [164]:
model.compile(optimizer='rmsprop',
    loss="binary_crossentropy",
    metrics=["accuracy"])

In [165]:
model.fit(X_train,y_train,batch_size=1000,steps_per_epoch=10,epochs=10)

Train on 20252 samples
Epoch 1/10
 9000/20252 [============>.................] - ETA: 0s - loss: 0.4614 - accuracy: 0.7794

In [166]:
model.evaluate(X_test,y_test,verbose=0)

[0.4702232706845554, 0.7806516]

In [167]:
model.predict(X_test[:10])

array([[0.27641365],
       [0.77756953],
       [0.8595767 ],
       [0.08840095],
       [0.19492865],
       [0.9689015 ],
       [0.52457434],
       [0.09915132],
       [0.8781318 ],
       [0.14730524]], dtype=float32)

In [168]:
y_test

array([1, 1, 1, ..., 1, 0, 0])

In [169]:
name="naman"

letters_only = re.sub(r'\d+' , # Search for all non-letters
                          "",          # Replace all non-letters with spaces
                          str(name))

#tokenizer = RegexpTokenizer(r'\w+')
#tokenizer.tokenize(name)

gen=[vocab[index] for index in name.lower() if index in vocab]
gen.insert(0,1)
gen.append(2)
f_name=gen

f_name

f_name=sequence.pad_sequences([f_name],maxlen=15)

model.predict(f_name)

array([[0.54389125]], dtype=float32)

IMP

In [ ]:
r"\^w+"   ### used to remove puntaution
r'\d+'     ## used for remove number